# Features extractor

## Read data

In [13]:
import pandas as pd
import numpy as np
import datetime
datasets_fake = ["FSF", "INT", "TWT"]
datasets_hum = ["TFP", "E13"]
datasets = datasets_hum + datasets_fake
folder_datasets = "./datasets/"
file_users = "users.csv"
file_tweets = "tweets.csv"
file_friends = "friends.csv"
file_followers = "followers.csv"

In [5]:
users = pd.read_csv(folder_datasets + datasets[0] + '/' + file_users)
tweets = pd.read_csv(folder_datasets + datasets[0] + '/' + file_tweets)
friends = pd.read_csv(folder_datasets + datasets[0] + '/' + file_friends)
followers = pd.read_csv(folder_datasets + datasets[0] + '/' + file_followers)
for dataset in datasets[1:]:
    pd.concat([users, pd.read_csv(folder_datasets + dataset + '/' + file_users)])
    pd.concat([users, pd.read_csv(folder_datasets + dataset + '/' + file_tweets)])
    pd.concat([users, pd.read_csv(folder_datasets + dataset + '/' + file_friends)])
    pd.concat([users, pd.read_csv(folder_datasets + dataset + '/' + file_followers)])
    
    

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
users.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,...,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset
0,24503,Roberto Bonanzinga,Bonanzinga,4340,5055,1466,267,256,Mon Nov 27 06:55:12 +0000 2006,http://t.co/32VHs0bxbY,...,99CC33,http://a0.twimg.com/profile_background_images/...,352726,D02B55,-28800.0,NaN,NaN,Partner at Balderton Capital (formerly Benchma...,2015-02-14 11:32:57,TFP
1,22903,effeffe,effeffe,164,132,194,12,4,Sun Nov 26 15:19:32 +0000 2006,http://t.co/CX7EHdk9DJ,...,EFEFEF,http://a0.twimg.com/images/themes/theme14/bg.gif,131516,009999,3600.0,NaN,NaN,L'uomo ha creato dio a sua immagine e somiglia...,2015-02-14 11:32:57,TFP
2,382393,Ciro Cattuto,ciro,1070,1154,832,871,92,Sun Dec 31 02:03:17 +0000 2006,http://t.co/26dNjSYe5Q,...,DAECF4,http://a0.twimg.com/images/themes/theme2/bg.gif,C6E2EE,1F98C7,3600.0,NaN,NaN,Research Director at ISI Foundation. Data scie...,2015-02-14 11:32:57,TFP
3,286543,Alessio Bragadini,abragad,6892,930,535,478,28,Wed Dec 27 14:55:17 +0000 2006,http://t.co/xoOdZi9jic,...,E0FF92,http://a0.twimg.com/profile_background_images/...,9AE4E8,0000FF,3600.0,NaN,NaN,Web and social media developer from Italy,2015-02-14 11:32:57,TFP
4,438023,fullcaffeine,fullcaffeine,2885,173,444,41,2,Tue Jan 02 09:01:50 +0000 2007,http://www.fullcaffeine.com,...,EFEFEF,http://a0.twimg.com/images/themes/theme14/bg.gif,131516,009999,3600.0,NaN,NaN,NaN,2015-02-14 11:32:57,TFP


In [7]:
users.loc[users['name'] == '']

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,...,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset


In [31]:
SAMPLE_SIZE = 10
for index, user in users.iterrows():
    #if index > SAMPLE_SIZE:
    #    break
    #print(user['name'])
    
    # Class A (Profile)
    #===================
    has_name = user['name'] not in ['NULL', 'NaN', '', ' ', pd.np.nan]
    has_image = user['default_profile_image'] != 1
    has_address = user['location'] not in ['NULL', 'NaN', '', ' ', pd.np.nan]
    has_biography = user['description'] not in ['NULL', 'NaN', '', ' ', pd.np.nan]
    followers_ge_30 = user['followers_count'] >= 30
    belongs_to_a_list = user['listed_count'] > 0
    nb_tweets_ge_50 = tweets.loc[tweets['user_id'] == user['id']].size >= 50
    url_in_profile = user['url'] not in ['NULL', 'NaN', '', ' ', pd.np.nan]
    followers_2_times_ge_friends = 2 * user['followers_count'] >= user['friends_count']
    
    bot_in_biography = type(user['description']) is str and 'bot' in user['description'].lower()
    ratio_friends_followers_around_100 = user['followers_count'] > 0 and 80.0 <= float(user['friends_count']) / user['followers_count'] >= 120.0
    duplicate_profile_picture = users.loc[users['default_profile_image'] == user['default_profile_image']].size > 1
    
    ratio_friends_followers_ge_50 = user['followers_count'] > 0 and float(user['friends_count']) / user['followers_count'] >= 50
    default_image_after_2_month = user['default_profile_image'] == 1 and (datetime.datetime.now() - datetime.datetime.strptime(user['created_at'],'%a %b %d %H:%M:%S +0000 %Y')) > datetime.timedelta(weeks=4) 
    friends_ge_100 = user['friends_count'] >= 100
    no_bio = user['description'] in ['NULL', 'NaN', '', ' ', pd.np.nan]
    no_location = user['location'] in ['NULL', 'NaN', '', ' ', pd.np.nan]
    no_tweets = tweets.loc[tweets['user_id'] == user['id']].size == 0
    
    nb_friends = user['friends_count']
    nb_tweets = tweets.loc[tweets['user_id'] == user['id']].size
    ratio_friends_followers_square = float(user['friends_count']) / pow(user['followers_count'], 2) if user['followers_count'] > 0 else 0
    
    age = (datetime.datetime.now() - datetime.datetime.strptime(user['created_at'],'%a %b %d %H:%M:%S +0000 %Y')).total_seconds()
    following_rate = float(user['friends_count']) / age

print("TOTAL accounts: " + str(users.size))

TOTAL accounts: 15946


In [8]:
users.isnull().sum()

id                                      0
name                                    0
screen_name                             0
statuses_count                          0
followers_count                         0
friends_count                           0
favourites_count                        0
listed_count                            0
created_at                              0
url                                   235
lang                                    0
time_zone                             101
location                              145
default_profile                       314
default_profile_image                 454
geo_enabled                           230
profile_image_url                       0
profile_banner_url                    249
profile_use_background_image           26
profile_background_image_url_https      0
profile_text_color                      0
profile_image_url_https                 0
profile_sidebar_border_color            0
profile_background_tile           